# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# !pip install gmaps

# !jupyter nbextension enable --py gmaps

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
csv_file = os.path.join("output_data/cities.csv")

city_weather = pd.read_csv(csv_file)
city_weather

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Bredasdorp,-34.5322,20.0403,62.60,94,58,5.75,ZA,1616557552
1,Władysławowo,54.7909,18.4009,41.00,81,90,8.05,PL,1616557552
2,Chapais,49.7834,-74.8492,37.40,52,90,4.61,CA,1616557553
3,Neuquén,-38.9516,-68.0591,69.80,46,0,11.50,AR,1616557553
4,Albany,42.6001,-73.9662,52.00,60,100,3.20,US,1616557425
...,...,...,...,...,...,...,...,...,...
574,Termoli,41.9888,14.9895,46.44,64,51,17.65,IT,1616557776
575,Montepuez,-13.1256,38.9997,68.54,98,99,4.72,MZ,1616557776
576,Atambua,-9.1061,124.8925,82.02,74,61,4.16,ID,1616557777
577,Balgazyn,51.0000,95.2000,37.24,67,68,4.45,RU,1616557777


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
gmaps.configure(api_key=g_key)

In [5]:
locations = city_weather[["Lat", "Lng"]]
humidity = city_weather["Humidity"]

fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False)
heat_layer.max_intensity = 100
heat_layer.point_radius = 5


fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
vacation_city = city_weather[(city_weather['Max Temp'] >= 70) & (city_weather['Max Temp'] <= 80)]
vacation_city = vacation_city[vacation_city['Wind Speed'] <= 10]
vacation_city = vacation_city[vacation_city['Cloudiness'] <= 10]
vacation_city = vacation_city[vacation_city['Humidity'] <= 70]
vacation_city = vacation_city.dropna()
vacation_city


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
11,Kapaa,22.0752,-159.3190,77.00,50,1,9.22,US,1616557353
142,Dubti,11.7361,41.0853,75.99,27,4,4.68,ET,1616557607
196,Makkah al Mukarramah,21.4267,39.8261,79.27,23,0,3.04,SA,1616557627
202,Buraidah,26.3260,43.9750,71.60,30,0,9.22,SA,1616557630
216,Dali,25.7000,100.1833,70.38,24,0,7.54,CN,1616557636
232,Barra de Tecoanapa,16.5167,-98.7500,75.74,66,0,5.41,MX,1616557643
279,Berbera,10.4396,45.0143,77.09,64,0,2.21,SO,1616557658
287,Salalah,17.0151,54.0924,71.60,49,0,3.44,OM,1616557662
317,Bandarbeyla,9.4942,50.8122,77.02,60,6,3.00,SO,1616557676
386,Nizwá,22.9333,57.5333,76.37,25,0,3.22,OM,1616557705


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
hotel_df = vacation_city
hotel_df['Hotel Name'] = ""
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
11,Kapaa,22.0752,-159.3190,77.00,50,1,9.22,US,1616557353,
142,Dubti,11.7361,41.0853,75.99,27,4,4.68,ET,1616557607,
196,Makkah al Mukarramah,21.4267,39.8261,79.27,23,0,3.04,SA,1616557627,
202,Buraidah,26.3260,43.9750,71.60,30,0,9.22,SA,1616557630,
216,Dali,25.7000,100.1833,70.38,24,0,7.54,CN,1616557636,
232,Barra de Tecoanapa,16.5167,-98.7500,75.74,66,0,5.41,MX,1616557643,
279,Berbera,10.4396,45.0143,77.09,64,0,2.21,SO,1616557658,
287,Salalah,17.0151,54.0924,71.60,49,0,3.44,OM,1616557662,
317,Bandarbeyla,9.4942,50.8122,77.02,60,6,3.00,SO,1616557676,
386,Nizwá,22.9333,57.5333,76.37,25,0,3.22,OM,1616557705,


In [8]:
for index, row in hotel_df.iterrows():
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    params = {
    "keyword": "hotel",
    "radius": 5000,
    "key": g_key,
    "location": f"{row['Lat']},{row['Lng']}", # These two are the variables different for each city that we will loop through
    }


    hotel_data = requests.get(base_url, params=params).json()
    try: 
        hotel_df.loc[index, "Hotel Name"] = hotel_data["results"][0]["name"]
        
    except IndexError:
        
        hotel_df.loc[index, "Hotel Name"] = "NaN"

# drop rows whose Hotel Name is NaN
hotel_df = hotel_df[hotel_df["Hotel Name"] != "NaN"]   
hotel_df 

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
11,Kapaa,22.0752,-159.3190,77.00,50,1,9.22,US,1616557353,The ISO
196,Makkah al Mukarramah,21.4267,39.8261,79.27,23,0,3.04,SA,1616557627,Pullman Zamzam Makkah
202,Buraidah,26.3260,43.9750,71.60,30,0,9.22,SA,1616557630,فندق تاورز HOTELTOWERS
216,Dali,25.7000,100.1833,70.38,24,0,7.54,CN,1616557636,The Jade Emu International Hostel
232,Barra de Tecoanapa,16.5167,-98.7500,75.74,66,0,5.41,MX,1616557643,Hotel Garapacho
279,Berbera,10.4396,45.0143,77.09,64,0,2.21,SO,1616557658,Damal Hotel
287,Salalah,17.0151,54.0924,71.60,49,0,3.44,OM,1616557662,HYATT SALALAH HOTEL
317,Bandarbeyla,9.4942,50.8122,77.02,60,6,3.00,SO,1616557676,JABIR HOTEL
386,Nizwá,22.9333,57.5333,76.37,25,0,3.22,OM,1616557705,Nizwa Residence Hotel Apartments
463,Acapulco de Juárez,16.8634,-99.8901,74.98,62,0,5.88,MX,1616557736,Dreams Acapulco Resort & Spa


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [10]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers) 
fig


# Display figure


Figure(layout=FigureLayout(height='420px'))